[View in Colaboratory](https://colab.research.google.com/github/connected-bsamadi/colab-notebooks/blob/master/Downloading_Kaggle_Datasets_in_Colab.ipynb)

# Training Dataset
Before running this notebook, go to "My Account" on Kaggle and press "Create New API Token", and upload the kaggle.json file to your Google Drive.

reference: [This notebook provides a recipe for using Kaggle API with Colab](https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF)

# Installing Kaggle API

In [0]:
!pip install kaggle >/dev/null

In [0]:
from googleapiclient.discovery import build
import io, os
import zipfile
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth, files
import pandas as pd

auth.authenticate_user()

In [0]:
# Find the kaggle.json on Google Drive
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

In [17]:
# Write kaggle.json to ~/.kaggle/
filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [18]:
!kaggle datasets list

ref                                                      title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
center-for-policing-equity/data-science-for-good         Data Science for Good: Center for Policing Equity    6MB  2018-10-02 17:38:13           1048  
lava18/google-play-store-apps                            Google Play Store Apps                               2MB  2018-09-18 20:49:49           6467  
blastchar/telco-customer-churn                           Telco Customer Churn                               172KB  2018-02-23 18:20:00           7557  
jenny18/honey-bee-annotated-images                       Annotated Honey Bee Images                          50MB  2018-09-16 23:42:12            588  
lucidlenn/sloan-digital-sky-survey                       Sloan Digital Sky Survey RD14  

# Downloading the Dataset

In [0]:
dataset = 'jigsaw-toxic-comment-classification-challenge'

In [0]:
folder = '/content/'+dataset
if not os.path.isdir(folder):
  os.mkdir(folder)
os.chdir(folder)
os.system('kaggle competitions download -c '+dataset)
for file in os.listdir():
  filename, file_extension = os.path.splitext(file)
  if file_extension == '.zip':
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [21]:
train_df=pd.read_csv('train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Converting the dataset to the Google Cloud AutoML CSV format

In [0]:
# Remove newline characters
def remove_newline(txt):
  txt_maxlen = 10000-2
  if len(txt) > txt_maxlen:
    qtxt = txt[0:txt_maxlen]
  else:
    qtxt = txt
  qtxt = '\"'+qtxt.replace('\"',' ')+'\"'
  qtxt = qtxt.replace('\r',' ')
  qtxt = qtxt.replace('\n',' ')
  qtxt = qtxt.replace('\t',' ')
  qtxt = qtxt.replace('#',' ')
  return qtxt

In [23]:
labels=[i for i in train_df][2:]
labels

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
automl_filename = dataset.replace('-','_')+'.csv'
automl_file = open(automl_filename,'wt')
for index, item in train_df.iterrows():
  line = remove_newline(item['comment_text'])
  safe=1
  for tag in labels:
    if item[tag]:
      line=line+','+tag
      safe=0
  if safe:
    line=line+',safe'
  line=line+'\n'
  automl_file.write(line)
automl_file.close()

In [0]:
files.download(automl_filename)

In [26]:
os.system('rm '+automl_filename)

0